### Import Libraries

In [3]:
import pathlib
import joblib
import importlib

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pycytominer.cyto_utils import DeepProfiler_processing


### Set Load/Save Paths

In [4]:
# paths to load merged features,index, and annotations from
merged_features_save_path = pathlib.Path(
    "/media/roshankern/63af2010-c376-459e-a56e-576b170133b6/data/cell-health-nuc-merged/"
)
dp_index_path = pathlib.Path(
    "/media/roshankern/63af2010-c376-459e-a56e-576b170133b6/data/cell-health-nuc-DP/inputs/metadata/index.csv"
)
annotations_path = pathlib.Path("../0.image-download/manifest/idr0080-screenA-annotation.csv")

# path to save normalized merged features to
normalized_merged_features_save_path = pathlib.Path(
    "/media/roshankern/63af2010-c376-459e-a56e-576b170133b6/data/cell-health-nuc-merged-normalized/"
)
normalized_merged_features_save_path.mkdir(parents=True, exist_ok=True)
scaler_save_dir = pathlib.Path("normalization-scalers/")
scaler_save_dir.mkdir(parents=True, exist_ok=True)

### Normalize merged single-cell data

In [5]:
for merged_single_cell_plate_path in merged_features_save_path.iterdir():
    # get only plate name from merged features file name
    plate = merged_single_cell_plate_path.name.split("-")[0]
    print(f"Normalizing plate {plate}...")
    
    # REMOVE LATER
    normalized_merged_plate_single_cells_save_path = pathlib.Path(f"{normalized_merged_features_save_path}/{plate}-normalized-merged-single-cell.csv")
    if normalized_merged_plate_single_cells_save_path.is_file():
        continue
    
    # load plate single-cell data
    print(f"Loading single-cell data...")
    #plate_merged_single_cells = pd.read_csv(merged_single_cell_plate_path, compression="gzip")
    # load uncompressed version REMOVE LATER AND USE ABOVE
    plate_merged_single_cells = pd.read_csv(merged_single_cell_plate_path)
    
    # create per-plate normalization scaler from the normalization population
    print(f"Deriving normalization scaler...")
    # find all cells that have had no reagent applied
    negative_control_single_cells = plate_merged_single_cells.loc[plate_merged_single_cells['Metadata_Reagent'] == "no reagent"]
    # get features for these negative control cells
    feature_cols = [col for col in negative_control_single_cells.columns.to_list() if "P__" in col]
    negative_control_feature_data = negative_control_single_cells[feature_cols].values
    # fit normalization scaler
    plate_scaler = StandardScaler()
    plate_scaler.fit(negative_control_feature_data)
    # save normalization scaler
    scaler_save_path = pathlib.Path(f"{scaler_save_dir}/{plate}-merged-normalization-scaler.joblib")
    joblib.dump(plate_scaler,scaler_save_path)
    
    # apply scaler to all single cell feature data
    print(f"Applying normalization scaler...")
    # get normalized feature data
    features = plate_merged_single_cells[feature_cols].values
    features = plate_scaler.transform(features)
    features = pd.DataFrame(features, columns=feature_cols)
    # get metadata for all single cells
    metadata_cols = [col for col in plate_merged_single_cells.columns.to_list() if "P__" not in col]
    metadata = plate_merged_single_cells[metadata_cols]
    
    # combine metadata and normalized features for all single cells (replace other single cell dataframe to not keep two dfs in memory)
    plate_merged_single_cells = pd.concat([metadata, features], axis=1)
    
    # compress and save merged single-cell data
    # print(f"Saving normalized features...")
    # normalized_merged_plate_single_cells_save_path = pathlib.Path(f"{normalized_merged_features_save_path}/{plate}-normalized-merged-single-cell.csv.gz")
    # plate_merged_single_cells.to_csv(normalized_merged_plate_single_cells_save_path, compression="gzip", index=False)
    
    # save merged single-cell data REMOVE LATER AND USED COMPRESSED VERSION ABOVE
    print(f"Saving normalized features...")
    normalized_merged_plate_single_cells_save_path = pathlib.Path(f"{normalized_merged_features_save_path}/{plate}-normalized-merged-single-cell.csv")
    plate_merged_single_cells.to_csv(normalized_merged_plate_single_cells_save_path, index=False)
    

plate_merged_single_cells

Normalizing plate SQ00014617...
Loading single-cell data...
